# The Batte of Neighborhoods

## Introduction, Business Problem

We will focus on Paris the French capital. As Paris is a great touristic French city, a lot of people come to visit it. Here you will see bars, restaurants and more. Also, Paris as a population density (inhabitants per km2) of 20.781,1. Indeed there is a large public and it will consequently be rewardful.
Now we need to know the data required.

## Data required

Multiple questions can be asked : Does restaurant with unusual meals really work ? What kind of restaurant should I open ? Does it really has an influence ?
For those questions we will not answer. We will focus on the activity and the income of an area. We will need to find nearby good restaurants having goos ratings.

The aim of this project is to solve the problem so we will need something that works.

What data will we need :
* Boroughs
* Neighborhoods
* Locations
* Venues (locations, category)
* Maybe some demographics can be interesting
* Income (average)

We will find these data online and some using foursquare API.